## 5.3 트리의 앙상블

목적: 앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 실습을 통해 학습

### 정형 데이터와 비정형 데이터

- 정형 데이터(structured data): 어떤 구조로 되어 있는 자료로 엑셀이나 csv, 데이터베이스에 저장하기 쉬움
- 비정형 데이터(unstructured data): 책의 글, 디지털카메라로 찍은 사진, 핸드폰으로 듣는 디지털 음악 등이 이에 속함

-> 정현 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 <strong>앙상블 학습(ensemble learning)</strong> 이라고 함

### 랜덤 포레스트(Random Forest)

랜덤 포레스트는 랜덤하게 샘플을 추출하여 훈련 데이터를 만드는데, 이때 한 샘플이 중복되어 추출될 수도 있다.

예를 들어, 1,000개 샘플이 들어 있는 가방에서 100개의 샘플을 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣는 식이다.

이렇게 만들어지 데이터 샘플을 <strong>부트스트랩 샘플(bootstrap sample)</strong> 이라고 한다.

다시 말해, 부트스트랩은 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 의미한다.

In [4]:
# 와인 분류하는 문제에 적용

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine   = pd.read_csv("https://bit.ly/wine_csv_data")
data   = wine[["alcohol", "sugar", "pH"]].to_numpy()
target = wine["class"].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [6]:
#  cross_validate()를 사용하여 교차 검증 수행

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf     = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

In [7]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9973541965122431 0.8905151032797809


In [8]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


RandomForesetClassifier에는 자체적으로 모델을 평가할 수 있는데, 

평가할 때 부트스트랩 샘플에 포함되지 않고 남은 샘플, 즉 OOB(out of Bag) 샘플을 가지고 평가할 수 있다.

In [10]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_) # 이를 사용하면 교차 검증 대신 사용 가능 -> 훈련 세트에 더 많은 샘플을 사용할 수 있음

0.8934000384837406


### 엑스트라 트리(Extra tree)

엑스트라 트리는 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용한다.

랜덤포레스트와 엑스트라 트리의 차이점은 부트스트랩을 사용하지 않는다는 점이다. 즉 각 결정 트리를 만들 때 전체 훈련 세트를 사용한다.

대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다.

<br>

하나의 결정 트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만 많은 트리를 앙상블 하기 때문에 과대적합을 막고 점증 세트의 점수를 높이는 효과가 있다.

In [11]:
from sklearn.ensemble import ExtraTreesClassifier

et     = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

In [14]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9974503966084433 0.8887848893166506


보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 한다.

하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 엑스트라 트리의 장점이다.

In [16]:
# 특성 중요도
# 알코올 도수, 당도, pH 순서

et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


### 그레이디언트 부스팅(Gradient Boosting)

그레디언트 부스틍은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이다.

사이킷런에서는 기본적으로 깊이가 3인 결정 트리를 100개 사용한다.

깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

gb     = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [21]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.8881086892152563 0.8720430147331015


위 결과는 과대적합이 되지 않았다. 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다.

학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있다.

In [26]:
# 결정 트리 수 : 500, 학습률 : 0.2 (기본 0.1)
gb     = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) 
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

In [27]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9464595437171814 0.8780082549788999


In [28]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


### 히스토그램 기반 그레이디언트 부스팅(Histogram-based Graident Boosting)

히스토그램 기반 그레이디언트 부스팅은 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘이다.

히스토그램 기반 그레이디언트 부스팅은 번저 입력 특성을 256개의 구간으로 나눈다. 따라서 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있다.

또한, 256개의 구간 중에 하나를 떼어 높고 누락된 값을 위해서 사용한다.

따라서 입력에 누락된 특성이 있떠라도 이를 따로 전처리할 필요가 없다.

In [33]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb    = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)

In [35]:
print(np.mean(scores["train_score"]), np.mean(scores["test_score"]))

0.9321723946453317 0.8801241948619236


In [36]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)

In [38]:
print(result.importances_mean) # 특성의 중요도, 평균, 표준편차

[0.08876275 0.23438522 0.08027708]


In [39]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)

In [40]:
print(result.importances_mean) # 특성의 중요도, 평균, 표준편차

[0.05969231 0.20238462 0.049     ]


In [41]:
print(hgb.score(test_input, test_target))

0.8723076923076923


그밖에 히스토그램 기반 알고리즘은 여럿 있는데 대표적으로 

XGBoot 와  MS 만든 LightGBM 이 있다. 